In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
# get root repository path
a = !pwd
repo_root = a[0].rsplit('ProGraML', maxsplit=1,)[0] + 'ProGraML'
print(repo_root)
#insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, repo_root)


In [ ]:
from pathlib import Path
import pickle

import numpy as np
import tqdm
import torch
from torch_geometric.data import Data, DataLoader, InMemoryDataset


### install pytorch-geometric
and other packages if needed, following: https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html

In [ ]:
from google.protobuf import text_format
from deeplearning.ml4pl.graphs import programl
from deeplearning.ml4pl.graphs.labelled import graph_tuple
from labm8.py import app

#FLAGS = app.FLAGS


In [ ]:
from deeplearning.ml4pl.graphs.unlabelled.llvm2graph import graph_builder

builder = graph_builder.ProGraMLGraphBuilder()

# Graph Tuple Legend


#### ```adjacencies: np.array```

A list of adjacency lists, one for each flow type, where an entry in an
adjacency list is a <src,dst> tuple of node indices.

`Shape (edge_flow_count, edge_count, 2), dtype int32`


#### ```edge_positions: np.array```

A list of edge positions, one for each edge type. An edge position is an
integer in the range 0 <= x < edge_position_max.

`Shape (edge_flow_count, edge_count), dtype int32`



#### ```node_x: np.array```

A list of node feature arrays. Each row is a node, and each column is an
feature for that node.

`Shape (node_count, node_x_dimensionality), dtype int32`


#### ```node_y: Optional[np.array] = None```

(optional) A list of node labels arrays.

`Shape (node_count, node_y_dimensionality), dtype float32`


#### ```graph_x: Optional[np.array] = None```

(optional) A list of graph features arrays.

`Shape (graph_x_dimensionality) OR (graph_count, graph_x_dimensionality) if
graph_count > 1, dtype int32`



#### ```graph_y: Optional[np.array] = None```

(optional) A vector of graph labels arrays.

`Shape (graph_y_dimensionality) OR (graph_count, graph_y_dimensionality) if
graph_count > 1, dtype float32`


## Disjoint graph properties

#### ```disjoint_graph_count: int = 1```

The number of disconnected graphs in the tuple.


#### ```disjoint_nodes_list: np.array = None```

A list of integers which segment the nodes by graph. E.g. with a GraphTuple
of two distinct graphs, both with three nodes, nodes_list will be
[0, 0, 0, 1, 1, 1].
`Shape (node_count), dtype int32:`


In [ ]:
# proto example
from google.protobuf import text_format
from deeplearning.ml4pl.graphs import programl_pb2 as proto

program_graph = proto.ProgramGraph()

with open('40.txt_9G8XzpcFlK.programl_proto', 'r') as f:
    proto = f.read()

proto = text_format.Parse(proto, program_graph)

In [ ]:
# from .ll example
with open('71.ll', 'r') as f:
    ll = f.read()

nx_graph = builder.Build(ll)

In [ ]:
# tuple example
graph_tup = graph_tuple.GraphTuple.CreateFromNetworkX(nx_graph)
print([a for a in dir(graph_tup) if '__' not in a])
print('\n')
print('edge_positions', [x.shape for x in graph_tup.edge_positions])

In [ ]:
# data example
edge_indices = [torch.from_numpy(a).to(dtype=torch.long) for a in graph_tup.adjacencies] # list of <M_i, 2>
edge_types = [i * np.ones_like(edge_indices[i])[:,0] for i in range(len(edge_indices))]
edge_attr = np.hstack(edge_types)
print('edge_attr', edge_attr.shape)

t_attr = torch.from_numpy(edge_attr).to(dtype=torch.long).view(-1,1)
print(t_attr.size())
print('edge_indices', [x.shape for x in edge_indices])
print('edge_types', [x.shape for x in edge_types])

#edge_pos = [torch.from_numpy(a).to(dtype=torch.long) for a in graph_tup.edge_positions]
print('graph_tup.edge_positions', [x.shape for x in graph_tup.edge_positions])
edge_pos = np.hstack(graph_tup.edge_positions)

edge_attr = np.vstack([edge_attr, edge_pos])

edge_attr = torch.from_numpy(edge_attr.T).to(torch.long)
print('edge_attr', edge_attr.size())

# node_x as data.x
x = torch.from_numpy(graph_tup.node_x).to(torch.long) #
y = torch.from_numpy(np.array(42)).to(torch.long).view(1)
print('x', x.size())
print('y', y.size())

In [ ]:
def tuple2data(graph_tup, class_label):
    # edges as data.edge_index
    # list of <M_i, 2> tensors  (M_i = num_edges for ith edge type)
    edge_indices = [torch.from_numpy(a).to(dtype=torch.long) for a in graph_tup.adjacencies] # list of <M_i, 2>

    edge_index = torch.cat(edge_indices, dim=0).t().contiguous() # <2, M>

    # (edge_type, edge_position) as data.edge_attr of shape <M, 2>
    edge_types = [i * np.ones_like(edge_indices[i])[:,0] for i in range(len(edge_indices))]
    edge_types = np.hstack(edge_types)
    edge_pos = np.hstack(graph_tup.edge_positions)
    edge_attr = np.vstack([edge_types, edge_pos]).T
    
    edge_attr = torch.from_numpy(edge_attr).to(dtype=torch.long).contiguous()  # <M, 2>
    
    assert edge_attr.size()[0] == edge_index.size()[1], f'edge_attr={edge_attr.size()} size mismatch with edge_index={edge_index.size()}'
    
    # node_x as data.x
    x = torch.from_numpy(graph_tup.node_x).to(torch.long)  # <N, 1>
    # class label as y
    y = torch.from_numpy(np.array(class_label)).to(torch.long).view(1)  # <1>

    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)
    return data

In [ ]:
# tuple2data example
tuple2data(graph_tup, 42)

In [ ]:
from multiprocessing import Pool
import tqdm
import os

def dump(outfile, data, mkdir=True):
    if mkdir:
        outfile.parent.mkdir(exist_ok=True, parents=True)
    with open(outfile, 'wb') as f:
        pickle.dump(data, f)

def load(file):
    with open(file, 'rb') as f:
        data = pickle.load(f)
    return data

In [ ]:
def preprocess_ds_raw_dir(ds_base, dump_nx=True, dump_tuple=True, dump_data=True, pool_size=12):
    """Preprocess all .ll files in folders named by class ids e.g. ds_base/1, ds_base/2, ...
    into pytorch-geometric data.Data instances.
    
    The intermediate nx graphs and graph_tuples can be saved as well.
    """

    out_base = ds_base.parent / (ds_base.name + '_programl')
    out_base.mkdir(parents=True, exist_ok=True)
    problems = open(out_base / 'problems.txt', 'a')
    print(f"=== DATASET {ds_base}: preprocessing will be saved in {out_base}")

    # get all subfolders 1/ 2/ etc
    folders = [x for x in ds_base.glob('*') if x.is_dir()]

    for i, folder in enumerate(folders):
        try:
            int(folder.name)
        except ValueError as e:
            print(f"Folder {i} has to be named with integervalues, but is {folder.name}.")
            raise e
    
    # multiprocessed loop over folders
    pool = Pool(processes=pool_size)
    task_args = [(folder, dump_nx, dump_tuple, dump_data, out_base) for folder in folders]
    
    for probs in tqdm.tqdm(pool.imap_unordered(_process_single_folder, task_args), total=len(task_args)):
        print(probs, file=problems)
    
    pool.close()
    pool.join()

    problems.close()
    print(f" * COMPLETED * === DATASET {ds_base}: preprocessing saved to {out_base}")
    
    

In [ ]:
def _process_single_folder(args):
    folder, dump_nx, dump_tuple, dump_data, out_base = args
    problems = ""
    
    print(f"=== Opening Folder {str(folder)} ===")
    
    label = int(folder.name)
    files = list(folder.glob('*.ll'))
    for i, file in enumerate(files):
        outfile_nx = out_base / '_nx' / folder.name / (file.name.rsplit('.', 1)[0] + '.nx.p')
        outfile_tuple = out_base / '_tuples' / folder.name / (file.name.rsplit('.', 1)[0] + '.tuple.p')
        outfile_data = out_base / folder.name / (file.name.rsplit('.', 1)[0] + '.data.p')

        # find out where to start processing

        # skip entirely?
        if outfile_data.is_file():
            continue

        # start at step 3: tuple -> data?
        if outfile_tuple.is_file():
            graph_tup = load(outfile_tuple)
            
            # ~~~ step 3: tuple --> data ~~~
            data = tuple2data(graph_tup, class_label=label)
            dump(outfile_data, data)
            continue

        # start at step 2: nx --> tuple ?
        if outfile_nx.is_file():
            nx_graph = load(outfile)
            
            # ~~~ step 2: nx --> tuple ~~~
            graph_tup = graph_tuple.GraphTuple.CreateFromNetworkX(nx_graph)
            if dump_tuple:
                dump(outfile_tuple, graph_tup)
            
            # step 3: tuple --> data            
            data = tuple2data(graph_tup, class_label=label)
            dump(outfile_data, data)
            continue
        
        # start in the beginning:
        # ~~~ step 1: .ll --> nx ~~~
        if i % 100 == 0:
            print(f"{folder.name} - [{i}/{len(files)}] Processing {str(file)} ...")
        
        with open(file, 'r') as f:
            bytecode = f.read()

        try:
            nx_graph = builder.Build(bytecode) # nx
            if dump_nx:
                dump(outfile, nx_graph)
        except:
            print(f"***** FAILING ON {str(file)} ... renaming file to .ll_ ")
            problems += str(file)
            problems += '\n'
            file.rename(file.with_suffix('.ll_'))
            continue

        # step 2: nx --> tuple
        graph_tup = graph_tuple.GraphTuple.CreateFromNetworkX(nx_graph)
        if dump_tuple:
            dump(outfile_tuple, graph_tup)

        # step 3: tuple --> data            
        data = tuple2data(graph_tup, class_label=label)
        dump(outfile_data, data)

    return problems

In [ ]:
# download dataset if needed
import wget
import zipfile

def download_and_unzip(url, dataset_name, data_folder):
    """
    Download and unzip data set folder from url
    :param url: from which to download
    :param dataset_name: name of data set (for printing)
    :param data_folder: folder in which to put the downloaded data
    """
    print('Downloading', dataset_name, 'data set...')
    if not os.path.exists(data_folder):
        os.makedirs(data_folder)
    data_zip = wget.download(url, out=data_folder)
    print('\tunzipping...')
    zip_ = zipfile.ZipFile(data_zip, 'r')
    assert os.path.isdir(data_folder), data_folder
    zip_.extractall(data_folder)
    zip_.close()
    print('\tdone')

def download_classifyapp(dataset_path):
    # get Path object
    if type(dataset_path) == str:
        dataset_path = Path(dataset_path)
    dataset_path = dataset_path / 'classifyapp_data'
        
    # Acquire data
    if not dataset_path.exists():
        dataset_path.mkdir(parents=True)
        download_and_unzip('https://polybox.ethz.ch/index.php/s/JOBjrfmAjOeWCyl/download',
                                      'classifyapp_data', str(dataset_path.absolute()))
    else:
        print(f'skipped downloading to {str(dataset_path.absolute())}')

# *needs action:*

In [ ]:
!pwd
!ls $(pwd)

In [ ]:
# Set where to store the dataset and download automagically
#ds_basepath = Path('/mnt/data/llvm/master_thesis_datasets')
ds_basepath = Path(repo_root) / 'deeplearning/ml4pl/poj104'
logs_basepath = ds_basepath / 'classifyapp_logs'

ds_basepath.mkdir(parents=True, exist_ok=True)
logs_basepath.mkdir(parents=True, exist_ok=True)

download_classifyapp(ds_basepath)

In [ ]:
# link those places into poj104 folder
data_source = str((ds_basepath / 'classifyapp_data').absolute())
print(data_source)
data_target = repo_root + '/deeplearning/ml4pl/poj104/classifyapp_data'
print(data_target)

logs_source = str(logs_basepath.absolute())
print(logs_source)
logs_target = repo_root + '/deeplearning/ml4pl/poj104/classifyapp_logs'
print(logs_target)

In [ ]:
! ln -s {data_source} {data_target}
! ln -s {logs_source} {logs_target}
! ls -lah {str(repo_root + '/deeplearning/ml4pl/poj104')} | grep classifyapp

In [ ]:
# start processing the smaller validation dataset
dataset_path = ds_basepath / Path('classifyapp_data/ir_val')
print(dataset_path.name)

preprocess_ds_raw_dir(dataset_path, pool_size=16)

In [ ]:
# train dataset
dataset_path = ds_basepath / Path('classifyapp_data/ir_train')
print(dataset_path.name)

preprocess_ds_raw_dir(dataset_path, pool_size=12)

In [ ]:
# test dataset
dataset_path = ds_basepath / Path('classifyapp_data/ir_test')
print(dataset_path.name)

preprocess_ds_raw_dir(dataset_path, pool_size=12)

# Build Pytorch-Geometric Dataset

In [ ]:
# naive dataloader that works on a list of data.Data instances in memory.

def get_dataloader(self, ds_base, config):
    ds_base = Path(ds_base) if type(ds_base) is str else ds_base
    datalist = []
    out_base = ds_base.parent / (ds_base.name + "_programl")
    print(f"=== DATASET {ds_base}: getting dataloader")

    folders = [x for x in out_base.glob("*") if x.is_dir() and x.name not in ['_nx', '_tuples']]
    for folder in tqdm.tqdm(folders):
        # skip classes that are larger than what config says to enable debugging with less data
        if int(folder.name) > config.num_classes:
            continue
        # print(f"=== Opening Folder {str(folder)} ===")
        for k, file in enumerate(folder.glob("*.data.p")):
            # print(f"{k} - Processing {str(file)} ...")
            with open(file, "rb") as f:
                data = pickle.load(f)
            datalist.append(data)
    print(f" * COMPLETED * === DATASET {ds_base}: returning dataloader")
    return DataLoader(datalist, batch_size=config.batch_size, shuffle=True)


In [ ]:
# better dataloader

import torch
from torch_geometric.data import InMemoryDataset

class POJ104Dataset(InMemoryDataset):
    def __init__(self, root, split, transform=None, pre_transform=None):
        self.split = split
        super().__init__(root, transform, pre_transform)

        if split == 'train':
            self.data, self.slices = torch.load(self.processed_paths[0])
        elif split == 'val':
            self.data, self.slices = torch.load(self.processed_paths[1])
        elif split == 'test':
            self.data, self.slices = torch.load(self.processed_paths[2])
        else:
            raise ValueError(f"split={split} but has to be one of train, val, test ")

    @property
    def raw_file_names(self):
        return 'classifyapp_data.zip' #['ir_val', 'ir_val_programl']
    
    @property
    def processed_file_names(self):
        return ['train_data.pt', 'val_data.pt'] #, 'test_data.pt']
    
    def download(self):
        # download to self.raw_dir
        print("I cannot yet download data automatically.")
    
    def process(self):
        # hardcoded
        num_classes = 104
        
        # output file
        if self.split == 'train':
            processed_path = self.processed_paths[0]
        elif self.split == 'val':
            processed_path = self.processed_paths[1]
        else:
            processed_path = self.processed_paths[2]

        # read data into huge `Data` list.
        data_list = []
        
        ds_base = Path(self.root)
        print(f'Creating {self.split} dataset at {str(ds_base)}')
        out_base = ds_base / ('ir_' + self.split + '_programl')
        assert out_base.exists(), f"{out_base} doesn't exist!"
        print(f"=== DATASET {out_base}: Collecting .data.p files into dataset")

        folders = [x for x in out_base.glob("*") if x.is_dir() and x.name not in ['_nx', '_tuples']]
        for folder in tqdm.tqdm(folders):
            # skip classes that are larger than what config says to enable debugging with less data
            if int(folder.name) > num_classes:
                continue
            for k, file in enumerate(folder.glob("*.data.p")):
                with open(file, "rb") as f:
                    data = pickle.load(f)
                data_list.append(data)
            
        print(f" * COMPLETED * === DATASET {out_base}: now pre-filtering...")
        # ...
        
        if self.pre_filter is not None:
            data_list = [d for d in data_list if self.pre_filter(d)]
        print(f" * COMPLETED * === DATASET {out_base}: Completed filtering, now pre_transforming...")
        
        if self.pre_transform is not None:
            data_list = [self.pre_transform(d) for d in data_list]
        
        data, slices = self.collate(data_list)
        torch.save((data, slices), processed_path)
        
    


In [ ]:
raw_data_path = repo_root + '/deeplearning/ml4pl/poj104/classifyapp_data'
print(raw_data_path)

In [ ]:
!ls {raw_data_path}

In [ ]:
dataset = POJ104Dataset(root=raw_data_path, split='val')

In [ ]:
dataset = POJ104Dataset(root=raw_data_path, split='train')

# tbd

In [ ]:
#class POJ104(InMemoryDataset):
#    torch_geometric.data.InMemoryDataset

In [ ]:
dataset.data